In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set=train_datagen.flow_from_directory(r'D:\Datasets\FaceMaskDetector-master\train',target_size=(120,120),batch_size=40,class_mode='binary')


test_datagen=ImageDataGenerator(rescale=1/255)
testing_set=test_datagen.flow_from_directory(r'D:\Datasets\FaceMaskDetector-master\test',target_size=(120,120),batch_size=40,class_mode='binary')




Found 1318 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [3]:
cnn=tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=38,kernel_size=3,activation='relu',input_shape=[120,120,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Conv2D(filters=38,kernel_size=3,activation='relu',input_shape=[120,120,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
cnn.fit(x=training_set,validation_data=testing_set,epochs=15)


#if result[0][0]==0:
    #print('With mask')
#else:
    #print('without mask')


Epoch 1/15
33/33 [==============================] - 73s 2s/step - loss: 0.5315 - accuracy: 0.7079 - val_loss: 0.1626 - val_accuracy: 0.9433
Epoch 2/15
33/33 [==============================] - 39s 1s/step - loss: 0.1946 - accuracy: 0.9294 - val_loss: 0.1368 - val_accuracy: 0.9588
Epoch 3/15
33/33 [==============================] - 32s 977ms/step - loss: 0.1186 - accuracy: 0.9598 - val_loss: 0.1056 - val_accuracy: 0.9742
Epoch 4/15
33/33 [==============================] - 28s 852ms/step - loss: 0.1032 - accuracy: 0.9636 - val_loss: 0.1122 - val_accuracy: 0.9742
Epoch 5/15
33/33 [==============================] - 29s 863ms/step - loss: 0.0893 - accuracy: 0.9674 - val_loss: 0.0847 - val_accuracy: 0.9794
Epoch 6/15
33/33 [==============================] - 29s 858ms/step - loss: 0.0841 - accuracy: 0.9727 - val_loss: 0.1783 - val_accuracy: 0.9485
Epoch 7/15
33/33 [==============================] - 28s 853ms/step - loss: 0.1076 - accuracy: 0.9621 - val_loss: 0.1279 - val_accuracy: 0.9536
Epoch

In [4]:
cnn.save('my_model.h5')


C:\Users\OM\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img,img_to_array
model=load_model('my_model.h5')
cascade=cv2.CascadeClassifier(r'D:\Datasets\FaceMaskDetector-master\haarcascade_frontalface_default.xml')
cap=cv2.VideoCapture(0)
while True:
    ret,img=cap.read()
    faces=cascade.detectMultiScale(img)
    for x,y,w,h in faces:
        faces1=img[y:y+h,x:x+w]
        cv2.imwrite('face.jpg',faces1)
        faces1=load_img('face.jpg',target_size=(120,120,3))
        faces1=img_to_array(faces1)
        faces1=np.expand_dims(faces1,axis=0)
        pred=model.predict(faces1)
        if pred<0.5:
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),3)
            cv2.putText(img,'Mask',(x,y),1,2,(255,255,0))
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'No Mask',(x,y),1,2,(255,255,0))
    cv2.imshow('Rahul',img)
    if cv2.waitKey(1)==13:
        break
cv2.destroyAllWindows()
cap.release()
    


1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 47ms/step
